# Goat Detection training

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets, models
from tqdm import tqdm
import cv2


In [27]:
class GoatDataset(Dataset):
    def __init__(self, size=64):
        self.size = size
        self.root = "./data/HoofedAnimals/img/"
        self.transforms = transforms.transforms.Resize(size)
        with open("./data/HoofedAnimals/ground_truth.txt") as f:
            content = f.readlines()
            self.labels = [line.split()[3] is not '0' for line in content]

    def __getitem__(self, index):
        path = self.root+str(index+1)+".pgm"
        im = cv2.imread(path)
        im = cv2.resize(im, (self.size, self.size))
        transform = transforms.ToTensor()
        im = transform(im)
        return im, torch.tensor(self.labels[index]).float()

    def __len__(self):
        return 200

dataset = GoatDataset()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

In [28]:
im, label = dataset[0]
print(label)
print(im)

tensor(0.)
tensor([[[0.2118, 0.3843, 0.2706,  ..., 0.4471, 0.3529, 0.3294],
         [0.3098, 0.3490, 0.3098,  ..., 0.3725, 0.4196, 0.4275],
         [0.4431, 0.4235, 0.1569,  ..., 0.3373, 0.3647, 0.2549],
         ...,
         [0.3020, 0.3255, 0.2824,  ..., 0.3451, 0.3216, 0.3490],
         [0.3059, 0.3569, 0.3961,  ..., 0.4000, 0.2784, 0.3647],
         [0.2863, 0.4314, 0.3255,  ..., 0.4078, 0.3647, 0.3961]],

        [[0.2118, 0.3843, 0.2706,  ..., 0.4471, 0.3529, 0.3294],
         [0.3098, 0.3490, 0.3098,  ..., 0.3725, 0.4196, 0.4275],
         [0.4431, 0.4235, 0.1569,  ..., 0.3373, 0.3647, 0.2549],
         ...,
         [0.3020, 0.3255, 0.2824,  ..., 0.3451, 0.3216, 0.3490],
         [0.3059, 0.3569, 0.3961,  ..., 0.4000, 0.2784, 0.3647],
         [0.2863, 0.4314, 0.3255,  ..., 0.4078, 0.3647, 0.3961]],

        [[0.2118, 0.3843, 0.2706,  ..., 0.4471, 0.3529, 0.3294],
         [0.3098, 0.3490, 0.3098,  ..., 0.3725, 0.4196, 0.4275],
         [0.4431, 0.4235, 0.1569,  ..., 0.3373,

In [51]:
class GoatDetection(nn.Module):
    def __init__(self):
        super(GoatDetection, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.final_layer = nn.Linear(1000, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        return self.sig(self.final_layer(self.resnet(x)))


In [55]:
bne = nn.BCELoss()
model = GoatDetection()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)

In [62]:
epochs = 1
for epoch in range(epochs):

  loop = tqdm(total=len(dataloader), position=0)
  total = 0
  for i, (image, label) in enumerate(dataloader):
    image, label = image.cuda(), label.cuda()
    optimizer.zero_grad()

    out = model(image)

    loss = bne(out, label)
    loss.backward()
    optimizer.step()
    total += loss.item()
    preds = (out >= 0.5).float().squeeze(1)
    
    accuracy = (preds == label).sum().item() / len(label)

    loop.set_description('epoch:{:d} loss:{:.4f} accuracy:{:.2f}'.format(epoch, total/(i+1), accuracy))
    loop.update(1)
  loop.close()


epoch:0 loss:1.4066 accuracy:0.70: 100%|██████████| 20/20 [00:02<00:00,  8.51it/s]


In [63]:
def prepare_ims(names, shared_root=""):
    tensors = []
    for n in names:
        path = shared_root+n
        im = cv2.imread(path)
        im = cv2.resize(im, (64, 64))
        transform = transforms.ToTensor()
        im = transform(im)
        im = im.unsqueeze(0)
        tensors.append(im)
#     print(tensors)
    return torch.cat(tensors, 0)

images = prepare_ims(["cow.jfif", "goat.jfif", "car.jfif"], shared_root="./data/")


In [64]:
print(images.size())
images = images.cuda()
out = model(images)
preds = (out >= 0.5).float().squeeze(1)
print(preds)

torch.Size([3, 3, 64, 64])
tensor([0., 0., 0.], device='cuda:0')
